In [3]:
import os
import random

In [ ]:
def generate_input_data(N=100, min_value=-100, max_value=100):
    """
    Создает файл с N строками случайных чисел.
    
    :param file_name: Имя файла для записи данных (по умолчанию "input_data.txt").
    :param N: Количество строк (по умолчанию 100).
    :param min_value: Минимальное значение для генерации чисел (по умолчанию -100).
    :param max_value: Максимальное значение для генерации чисел (по умолчанию 100).
    """
    # Открываем файл для записи
    with open("input_data.txt", "w") as file:
        for i in range(N):
            # Генерируем 4 случайных числа в диапазоне [min_value, max_value]
            numbers = [random.randint(min_value, max_value) for i in range(4)]
            # Записываем их в файл в виде строки, разделенной пробелами
            file.write(f"{numbers[0]} {numbers[1]} {numbers[2]} {numbers[3]}\n")

# Пример использования функции
generate_input_data(N=50)


In [ ]:
def generate_mem_file(N=10, min_value=-100, max_value=100, hex_width=32):
    import random

    with open("input_data.mem", "w") as file:
        for i in range(N):
            # Генерация 4 случайных чисел
            a = random.randint(min_value, max_value)
            b = random.randint(min_value, max_value)
            c = random.randint(min_value, max_value)
            d = random.randint(min_value, max_value)

            # Преобразование в шестнадцатеричный формат с фиксированной длиной
            a_hex = f"{a & 0xFFFFFFFF:08X}"
            b_hex = f"{b & 0xFFFFFFFF:08X}"
            c_hex = f"{c & 0xFFFFFFFF:08X}"
            d_hex = f"{d & 0xFFFFFFFF:08X}"


            # Запись в файл
            file.write(f"{a_hex} {b_hex} {c_hex} {d_hex}\n")

# Пример использования
generate_mem_file(N=10, hex_width=32)